<a href="https://colab.research.google.com/github/LilySu/Dungeon-Game/blob/master/Dugeon_Game_Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Wed Apr 29 23:40:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 259Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 70.4Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 291Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:04, 114Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 259Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 139Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 147Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [3]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
file_name = "dungeon_name.txt.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [6]:
gpt2.copy_file_from_gdrive(file_name)

FileNotFoundError: ignored

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [7]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 425.52it/s]

Loading dataset...
dataset has 7951 tokens
Training...


[10 | 19.75] loss=2.22 avg=2.22
[20 | 32.22] loss=1.31 avg=1.76
[30 | 44.69] loss=0.80 avg=1.44
[40 | 57.18] loss=0.21 avg=1.13
[50 | 69.66] loss=0.09 avg=0.92
[60 | 82.16] loss=0.13 avg=0.78
[70 | 94.63] loss=0.03 avg=0.67
[80 | 107.12] loss=0.03 avg=0.59
[90 | 119.60] loss=0.03 avg=0.52
[100 | 132.06] loss=0.03 avg=0.47
[110 | 144.56] loss=0.03 avg=0.43
[120 | 157.05] loss=0.02 avg=0.39
[130 | 169.55] loss=0.03 avg=0.37
[140 | 182.04] loss=0.03 avg=0.34
[150 | 194.51] loss=0.03 avg=0.32
[160 | 207.00] loss=0.02 avg=0.30
[170 | 219.51] loss=0.02 avg=0.28
[180 | 232.00] loss=0.04 avg=0.27
[190 | 244.52] loss=0.02 avg=0.25
[200 | 257.01] loss=0.03 avg=0.24
======== SAMPLE 1 ========
 wolf spell, or the bond between a city and its protector, or the power of King or Country. Whatever your inspiration, you can create your own place of power.

Underground Cave
Level Cantrip

Description

You make a show of flashing around this cave. The GM will tell you what here is magical. You can even cr

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [1]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

NameError: ignored

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [5]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [8]:
gpt2.generate(sess, run_name='run1')

Gear
Your load is 9+Str. You start with dungeon rations (5 uses, 1 weight), leather armor (1 armor, 1 weight), 3 uses of your chosen poison, and 10 coins. Choose your arms:

Dagger (hand, 1 weight) and short sword (close, 1 weight)
Rapier (close, precise, 1 weight)
Choose a ranged weapon:

3 throwing daggers (thrown, near, 0 weight)
Ragged Bow (near, 2 weight) and bundle of arrows (3 ammo, 1 weight)
Choose one:

Adventuring gear (1 weight)
Healing potion (0 weight)
Bonds
Fill in the name of one of your companions in at least one:

I stole something from                         .

                         has my back when things go wrong.

                        knows incriminating details about me.

                        and I have a con running.

Advanced Moves
When you gain a level from 25, choose from these moves.

Cheap Shot
When using a precise or hand weapon, your backstab deals an extra +1d6 damage.

Cautious
When you use trap expert you always get +1 hold, even on a 6-.

Wea

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [6]:
gpt2.generate(sess,
              length=30,
              temperature=0.8,
              prefix="as you go into the room",
              nsamples=1,
              batch_size=1
              )

as you go into the room with the spell and its effects.

Spell Defense
You may end any ongoing spell immediately and use the energy of its dissipation to deflect an


In [7]:
generated_list = []
for i in range(30):
  result = gpt2.generate(sess,
              length=30,
              temperature=0.8,
              prefix="as you go into the room",
              nsamples=1,
              batch_size=1
              )
  generated_list.append(result)
generated_list

as you go into the room with these spells. They have some kind of danger in them, but the GM will give you one to four of the following conditions:

Its
as you go into the room with the spell and ask the GM any three questions.

Self-Powered
When you have time, arcane materials, and a safe space
as you go into the room with the spell.

The spells effects are maximized
The spells targets are doubled
Fount of Knowledge
When you spout lore about
as you go into the room with the spell and ask these questions:

Is there a trap here and if so, what activates it?
What does the trap do when
as you go into the room with the spell. The GM will reveal the details of a grim portent to youa bleak event that will come to pass without your intervention. They
as you go into the room with the spell and ask these questions:

Is there a trap here and if so, what activates it?
What does the trap do when
as you go into the room with the spell.

The best you can do is a lesser version, unreliable and limite

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [0]:
with open('room_description.txt', 'w') as filehandle:
    filehandle.writelines("%s\n" % place for place in generated_list)

In [0]:
import json

# define list with values

# open output file for writing
with open('room_description.txt', 'w') as filehandle:
    json.dump(generated_list, filehandle)

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [17]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

FailedPreconditionError: ignored

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [18]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 234Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 71.2Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 302Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:48, 63.4Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 365Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 141Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 131Mit/s]                                                       


In [19]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

ValueError: ignored

In [20]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

ValueError: ignored

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.